# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [ ]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


In [ ]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


In [ ]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'N'
vuln = [False, False]  # fist element is NS, second element is EW

hands = ['87.84.T7.AKJ9764','JT53.QT.AJ6542.8','KQ96.K9.Q83.QT32','A42.AJ76532.K9.5']

auction = ['3C', 'PASS', '4C', '4H', 'PASS', 'PASS', 'PASS']

play = ['CA', 'C8', 'C3', 'C5',
    'S8', 'ST', 'SQ', 'SA',
    'DK', 'DT', 'D6', 'D3',
    'D9', 'D7', 'DA', 'D8', 
    'DJ', 'DQ', 'HJ', 'S7']

In [ ]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

In [ ]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

await card_by_card.analyze()

In [ ]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

the engine very confidently opens `1C` and doesn't even consider `1N`

In [ ]:
# what about the opening lead? let's see...

card_by_card.cards['CA'].to_dict()

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [ ]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)